# Notebook to create graph of players and teams

The resulting graph is a bipartite graph with players on one side and teams on the other.
Teams are additionally separated by season, meaning that every team node also has the season attached to it.

Importing all relevant libraries

In [104]:
import csv
import networkx as nx

Define csv file path

In [105]:
filepath = '../data/NBA Player Stats(1950 - 2022).csv'

Create a graph

In [106]:
graph = nx.Graph()

Read csv and add players and teams to it

In [107]:
with open(filepath, 'r', encoding="utf8") as file:
    csvreader = csv.reader(file)
    for row in csvreader:
        year = row[2]
        name = row[3]
        team = row[6]
        team_node_name = team + "-" + year
        if not graph.has_node(team_node_name):
            graph.add_node(team_node_name, type='team')
        if not graph.has_node(name):
            graph.add_node(name, type='player')
        graph.add_edge(name, team_node_name)

Mark nodes that represent star players

In [108]:
star_players = ['LeBron James', 'Kevin Durant']
# todo: dodaj preverjanje, kako oznaciti igralca kot star playerja

Create another graph that also holds information for start players

In [109]:
for node in graph.nodes():
    graph.nodes[node]['star_player'] = str(node in star_players)

Display network info and save the file

In [110]:
print(graph)
player_nodes = []
team_nodes = []
for (p, d) in graph.nodes(data=True):
    if d['type'] == 'player':
        player_nodes.append(p)
    elif d['type'] == 'team':
        team_nodes.append(p)
print('players =', len(player_nodes))
print('teams =', len(team_nodes))
nx.write_pajek(graph, '../data/graph.net', encoding='utf-8')

Graph with 6134 nodes and 28237 edges
players = 4487
teams = 1647
